In [1]:

#Uncomment following lines and edit path accordingly, if this script is not in same folder as the simtools folder
#import sys
#sys.path.insert(0, os.path.abspath('C:/path/to/folder_with_simtools-folder'))
import pickle
import os.path
import numpy as np
import pandas as pd
import tensorflow_addons as tfa
from math import exp
import glob
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
from zipfile import ZipFile
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
import pandas as pd 
import numpy as np 
import datetime
import tensorflow as tf
from tensorflow.keras import Model, Sequential, Input,models
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras import Input, Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanAbsoluteError
import os
import warnings
warnings.filterwarnings('ignore')
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight') 
# Above is a special style template for matplotlib, highly useful for visualizing time series data
%matplotlib inline
from pylab import rcParams
import statsmodels.api as sm
from numpy.random import normal, seed
from scipy.stats import norm
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.arima_model import ARIMA
import math
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import glob
from matplotlib.ticker import MultipleLocator,FormatStrFormatter,MaxNLocator 
from scipy.optimize import curve_fit
''' setting the global parameters of matplotlib to academic puplication quality settings ''' 
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
mpl.rcParams['font.family'] = 'Arial'
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}
mpl.rc('font', **font)
import matplotlib.colors as mcol
import matplotlib.cm as cm
from functionen import *

# Simulation Data 

In [82]:
# path for the ISA file for Column Name 
path_ISA_csv = r'C:\Users\AA Rig\Desktop\py too sim trnsys/ISA Seriensimulation.csv'


path_sim_data_inst = os.path.join(os.getcwd(),r'Simulation Data\DF System') #simulation data for instantaneous system
path_sim_data_str  = os.path.join(os.getcwd(),r'Simulation Data\Storage_System')   # simulation data for Storage System
Heizung_bes = os.path.join(os.getcwd(), r'Simulation Data\Bestandbau') 
Heizung_neu = os.path.join(os.getcwd(), r'Simulation Data\Neubau') #path to simulation data


In [78]:
path_sim_data_str

'C:\\Users\\AA Rig\\Desktop\\notebooks\\Simulation Data\\Storage_System'

In [84]:
'''
loading the simulation data into an Array of Dataframes

'''
buildings_inst = instaSystem_auswertung(path_ISA_csv,path_sim_data_inst)
buildings_str = storage_auswertung(path_ISA_csv,path_sim_data_str)
Heizung_bes_3 = Heizung_bestand_lader_long(Heizung_bes)   #load the simulation data for the heating system (convection heater)
Heizung_neu_2 = Heizung_neubau_lader(Heizung_neu) #loading the simulation data for Heating system (floor heating)


100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [00:05<00:00,  5.76it/s]


In [66]:
def map_temp_i(buildings_inst,Ge=0,step=2):
    '''
    this function maps The supply temperature with the index of Simulation runs in order to retrieve the specific Simulation run that repreasent the 
    the desired supply Temperature. returns a dictionary with temperature and Index. 
    
    '''
    i = get_iterator(buildings_inst,Ge = Ge,step = step)
    T = np.arange(70,130,1).tolist()
    #T= T+T;T = sorted(T);T
    keys_list = T
    values_list = i
    zip_iterator = zip(keys_list, values_list)
    a_dictionary = dict(zip_iterator)
    return a_dictionary

# Domestic Water (Insta System)


In [6]:

def predictor_inst(df,popt,func):
    
    """ 
    using the fit parameter to return a profile of return temperature.  
    Parameters:
    df: simulation run as dataframe
    popt: Fit Parameter
    func : the fit function, return temp = f(Q,supply temp,popt)
    Return type:return an array of Dataframes, each dataframe is a TRNSYS Simulation run. 
 

    """


    #provide the clustering according to circulation time 
    def zirkclust(df):                     
        df['cluster'] = df.index.hour
        hours = df['cluster'].values
        iterator = np.arange(len(hours))
        for i in tqdm(iterator) :

            time = hours[i]
            if time >= 5 and  time < 23:
                hours[i] = 1
            else :
                hours[i] = 0
        return hours

    cluster = zirkclust(df)
    df['cluster'] = cluster  
    
    #inputs for the function 
    Tvl = np.array(df['T_ah_hot_in'])
    dcw = np.array(df['dcw'])
    Q = np.array(df['dQ_DH']/df['dQ_DH'].max())
    y_data6 = np.array(df['T_ph_hot_out'])

    #setting up the profile 
    y_prediction6 = np.arange(len(df))      
    iterator = np.arange(len(df))
    
    #calculating the return temperature profile according to the parameter 
    for i in tqdm(iterator):
        if cluster[i] == 1:
            y_prediction6[i] = func((Q[i],Tvl[i]),popt[0],popt[1],popt[2],popt[3])  


        elif cluster[i] == 0:
            y_prediction6[i] = dcw[i]                                             #prediction when the circulation is turned off
    return y_prediction6


In [7]:


def prediktor_df(buildings_inst2,i,m):
    
    """ 
    this function uses the predictor_inst return and builds the final profile of the return temperature

    Parameters:
    buildings_str3: simulation data
    i: index of the simulation run
    m : parameter to choose between single family Building and Multi family Building 
    Return type:
    df : dataframe including the generated return temperatur with the column name y_prediction6
    accuracy : return an array fit goodnes measurments 
    
    Description : 
    call the predictor_inst function to generate the profile of a simulation run with the fit parameter then replace
    the return temperature predicted for the near zero heating power with the supply temperature and generate the mass flow rate profile 

    """
    buildings_inst2 = insert_dcw(buildings_inst2)  #insert the cold water profile 

    Data = resampler_3auf15(buildings_inst2) 
    null_index = null_leistung_index(Data,min_leistung_kw = 0.02) # find the index of the near zero heating power
    df = Data[i]
    def func(X, a, b, c,d):
        Q,Tvl = X
        return a*np.exp(-b*Q)*Tvl**c + d 

    cp = 4181/3600 #Wh /kg K    
    popt_efh = [ 5.65939827e+01,  2.36387058e+01, -1.02168853e-02, 1.37433361e+01]   #fit parameter for single family building 
    popt_mfh = [ 6.01166844e+02,  2.22532621e+01, -5.20360987e-01, 9.29918384e+00]   #fit parameter for multi family building

    #popt_efh = [56.59399414564164,1.0139398201218028,-0.010216912889541773,13.743329201972891]  #popt Leistung ohne standraization 
    #popt_mfh = [601.12578707499,0.6550009739560259,-0.5203463233928007,9.299040435860078]
    
    popt = [popt_efh,popt_mfh]
    y_prediction6 = predictor_inst(df,popt[m],func)                       #calculation of the prediction 


    df['y_prediction6'] = y_prediction6                                                
    df['y_prediction6'].loc[null_index[i]] = df['T_ah_hot_in']                          #postions with near zero heating power with 
                                                                                        # return temp = supply temmp
        
    df['dm_DH_zi'] = (df['dQ_DH']*1000)/(cp*(df['T_ah_hot_in']-df['y_prediction6']))    #calculation of the mass flow rate of the primery seit
                                                                                        #according to the equation mpr kg/hr = Q Kw * 1000 W/kW /cp Wh/kg K*(Tvlp-Trlp)K
    df['dm_DH_zi'].loc[null_index[i]] = 0                                                #points with near zero power with zero Massflow replacing
    
    
    MAE , MSE, r2,accuracy,ab = evaluate(np.array(df['T_ph_hot_out']),np.array(df['y_prediction6'])) #calculating the goodness of the fit
    df['ab'] = np.array(ab)
    accuracy[1] = r2
    return df,accuracy


In [8]:
EFH_map = map_temp_i(buildings_inst,Ge=0,step=2)
MFH_map = map_temp_i(buildings_inst,Ge=1,step=2)


In [10]:
#generating the final return temperature profile with the fit goodness
TWW_DF,accuracy  = prediktor_df(buildings_inst,i=EFH_map[70],m=0)


100%|████████████████████████████████████████████████████████████████████████| 35041/35041 [00:00<00:00, 479904.28it/s]

The model performance for testing set
--------------------------------------
MAE is 1.0763249747269414
MSE is 30.069054215201007
MAPE is 0.08115170281585396
R2 score is 0.7590757226566895
95% percent of the data lay between ± 4.12


# Domestic Water (Storage System)

In [11]:
def predictor_speicher(df,func,func2,popt,popt_2):
    
    """ 
    using the fit parameter to return a profile of return temperature.  
    Parameters:
    df: simulation run as dataframe 
    popt: Fit Parameter
    func : the fit function, return temp = f(Q,supply temp,popt)
    Return type:return an array of Dataframes, each dataframe is a TRNSYS Simulation run. 

    """
    
    Tvl = np.array(df['T_ah_hot_in'])
    dcw = np.array(df['dcw'])
    Q = df['dQ_DH']/df['dQ_DH'].max()
    '''a*Tin**b+'''


    y_prediction6 = func((Q,Tvl),popt[0],popt[1],popt[2],popt[3])

    df['y_prediction6'] = np.array(y_prediction6)

    # um funfuhr fix 
    df['hour'] = df.index.hour
    df['minute'] = df.index.minute
    m = df[df['minute'] == 0]
    h = m[m['hour'] == 5]
    funfhour = df.loc[h.index]
    Tvl = np.array(funfhour['T_ah_hot_in'])


    Tout = func2(Tvl,popt_2[0])

    df['y_prediction6'].loc[h.index] = Tout
    
    return df['y_prediction6']

In [12]:



def speicher_aufbau(buildings_str3,i=0,m=0):
    
    """ 
    this function uses the predictor_speicher return and builds the final profile of the return temperature

    Parameters:
    buildings_str3: simulation data
    i: index of the simulation run
    m : parameter to choose between single family Building and Multi family Building 
    Return type:
    df : dataframe including the generated return temperatur with the column name y_prediction6
    accuracy : return an array fit goodnes measurments 
    
    Description : 
    call the predictor_speicher function to generate the profile of a simulation run with the fit parameter then replace
    the return temperature predicted for the near zero heating power with the supply temperature and generate the mass flow rate profile 

    """
    
    cp = 4181/3600 #Wh /kg K
    buildings_str3 = insert_dcw(buildings_str3)
    Data = resampler_3auf15(buildings_str3)
    null_index = null_leistung_index(Data,min_leistung_kw = 0.02)


    #Data = null_leistung(Data,min_leistung_kw = 0.02)
    #Data = alles(Data,Data[0].columns,get_iterator(buildings_str3,Ge =1,step = 2))

    df = Data[i]

    def func(X, a, b, c,d):
        Q,Tvl = X
        return a*Tvl**b*Q**c+d

    def func2(X, a):
        Tvl = X
        return a*Tvl+(1-a)*35

    popt_efh = [ -29.28247137, 0.0602326899999,1.23415097, 58.86508069]
    popt_mfh = [ -45.11118121, 0.0602962899, 1.07408762, 58.47721043]
    #popt_efh = [-4.83020138,  0.06023261 , 1.23414613 ,58.8650969 ]
    #popt_mfh = [-2.79826629 , 0.06029634 , 1.07408762 ,58.47721042]
    
    popt_1 = [popt_efh,popt_mfh]

    popt_2_efh = [0.06018571]
    popt_2_mfh = [-0.02725702]

    popt_2 = [popt_2_efh,popt_2_mfh]

    y_prediction6 = predictor_speicher(df,func,func2,popt_1[m],popt_2[m])

    df['y_prediction6'] = y_prediction6
    df['y_prediction6'].loc[null_index[i]] = df['T_ah_hot_in']
    df['dm_DH_zi'] = (df['dQ_DH']*1000)/(cp*(df['T_ah_hot_in']-df['y_prediction6']))  
    df['dm_DH_zi'].loc[null_index[i]] = 0
    
    
    MAE , MSE, r2,accuracy,ab = evaluate(np.array(df['T_ph_hot_out']),np.array(df['y_prediction6']))
    df['ab'] = np.array(ab)
    accuracy[1] = r2
    
    
    return df,accuracy

In [15]:
TWW_SP,accuracy = speicher_aufbau(buildings_str,i=EFH_map[70],m=0)



100%|███████████████████████████████████████████████████████████████████████████████| 120/120 [00:00<00:00, 362.46it/s]


The model performance for testing set
--------------------------------------
MAE is 0.9549178441649306
MSE is 2.042149993217819
MAPE is 0.017600306274105477
R2 score is 0.9439008433983953
95% percent of the data lay between ± 1.65


# Heating (convection heater)  (Bestandsbau)

In [25]:
cp = 4181/3600 #Wh/kg.K 

In [26]:
def insert_hk(data):
    '''
    this function calculate the Heizkurve and insert it to the simulations data
    
    '''
    for i in tqdm(np.arange(len(data))):
        T_Mean = data[i]['T_amb'] 
        task32 = ((-3.597*T_Mean+65.4)**(1/1.3))-(0.33*T_Mean)+ 24 #traum = 18 
        #task32 = ((-3.250*T_Mean+63.6)**(1/1.3))-(0.31*T_Mean)+ 26.25 #T Gebäude = 20
        task32 = task32.fillna(task32.min())
        data[i]['T_ah_cold_out_zi'] =  task32
    return data

In [27]:
def heizung_bestand_predictor(df,func,popt):

    """ 
    using the fit parameter to return a profile of return temperature.  
    Parameters:
    df: simulation run as dataframe 
    popt: Fit Parameter
    func : the fit function, return temp = f(Q,supply temp, Heizkurve, popt)
    Return type:return an array of Dataframes, each dataframe is a TRNSYS Simulation run. 
 

    """
    x_data6 = np.array([df['dQ_DH'],df['T_ah_hot_in'],df['T_ah_cold_out_zi']])
    y_predicted = func(x_data6,popt[0],popt[1],popt[2],popt[3],popt[4])

    return y_predicted


In [28]:
path_ISA_csv = r'C:\Users\AA Rig\Desktop\py too sim trnsys/ISA Seriensimulation.csv'



In [29]:
buildings_efh,buildings_mfh =efh_mfh_seperator(Heizung_bes_3) #seprate the simulation data into two arrays single family and multiple family buildings 


100%|██████████████████████████████████████████████████████████████████████████████████| 60/60 [01:04<00:00,  1.08s/it]


In [30]:
buildings_efh = insert_hk(buildings_efh) 
buildings_mfh = insert_hk(buildings_mfh)

100%|█████████████████████████████████████████████████████████████████████████████████| 60/60 [00:00<00:00, 120.70it/s]


In [31]:
def Heizung_aufbau(buildings_efh,i =0,m =0 ):
    
    """ 
    this function uses the heizung_bestand_predictor return and builds the final profile of the return temperature

    Parameters:
    buildings_efh: simulation data single family Building and Multi family Building 
    i: index of the simulation run
    m : parameter to choose between single family Building and Multi family Building 
    Return type:
    df : dataframe including the generated return temperatur with the column name y_prediction6
    accuracy : return an array fit goodnes measurments 
    
    Description : 
    call the predictor_speicher function to generate the profile of a simulation run with the fit parameter then replace
    the return temperature predicted for the near zero heating power with the supply temperature and generate the mass flow rate profile 

    """
    Datei = resampler_3auf15(buildings_efh)
    null_index = null_leistung_index(Datei,min_leistung_kw = 0.02)

    #Datei = null_leistung(Datei,min_leistung_kw = 0.02)
    #Datei = alles(Datei,Datei[0].columns,get_iterator(buildings_efh,Ge = 0,step = 1))

    df = Datei[i]

    popt_efh = [ 0.51701225, -0.10933189,  0.55703713,  0.74078089, 19.09031136]
    popt_mfh = [ 0.06386505, -0.08838719,  0.07358941,  1.53155124, 14.3061995 ]
    #popt_efh = [ 0.15020795 ,-0.09421033 , 0.02485062,  1.36827334 ,15.92499546] 20 °C T raum 
    #popt_mfh = [ 0.17081797, -0.08995538 , 0.03143938 , 1.33981089 ,13.54041004] 20 °C T raum 
    popt =[popt_efh,popt_mfh]
    def func(X, a, b, c,d,e):
        Q,Tvl,Tamb = X
        return a*Tvl**b*Q**c*Tamb**d+e

    y_prediction6 = heizung_bestand_predictor(df,func,popt[m])
    df['y_prediction6'] = y_prediction6
    df['y_prediction6'].loc[null_index[i]] = df['T_ah_hot_in']
    df['dm_DH_zi'] = (df['dQ_DH']*1000)/(cp*(df['T_ah_hot_in']-df['y_prediction6']))  
    df['dm_DH_zi'].loc[null_index[i]] = 0

    MAE , MSE, r2,accuracy,ab = evaluate(np.array(df['T_ph_hot_out']),np.array(df['y_prediction6']))
    df['ab'] = np.array(ab)

    return df,accuracy

In [32]:
Heizung_bestand = Heizung_aufbau(buildings_efh,i =0,m =0 )

100%|████████████████████████████████████████████████████████████████████████████████| 60/60 [00:00<00:00, 1016.76it/s]

The model performance for testing set
--------------------------------------
MAE is 0.9407862792282954
MSE is 7.292936705454507
MAPE is 0.03318257668974864
R2 score is 0.9796128870482759
95% percent of the data lay between ± 3.34


# Heating System (floor heating) (Neubau) 

In [33]:
def insert_hk(data):
    for i in tqdm(np.arange(len(data))):
        T_Mean = data[i]['T_amb'] 
        task32 = -0.0036*T_Mean*T_Mean-0.3929*T_Mean+31
        data[i]['T_ah_cold_out_zi'] =  task32
    return data

In [36]:
buildings_efh_nb, buildings_mfh_nb =efh_mfh_seperator(Heizung_neu_2) 


100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [00:06<00:00,  4.45it/s]


In [37]:

buildings_efh_nb = insert_hk(buildings_efh_nb)
buildings_mfh_nb = insert_hk(buildings_mfh_nb)

100%|████████████████████████████████████████████████████████████████████████████████| 31/31 [00:00<00:00, 1347.50it/s]


In [39]:

def heizung_neubau_predictor(df,func,popt):
    
    """ 
    using the fit parameter to return a profile of return temperature.  
    
    Parameters:
    df: simulation run as dataframe 
    popt: Fit Parameter
    func : the fit function, return temp = f(Q,supply temp, Heizkurve, popt)
    
    Return type: profile of return temperature as an Array
    
    """

    Q = np.array(df['dQ_DH']/df['dQ_DH'].max())
    Tvl = np.array(df['T_ah_hot_in']/df['T_ah_hot_in'].max())
    Tvls = np.array(df['T_ah_cold_out_zi']/df['T_ah_cold_out_zi'].max())
    #Q = np.array(df['dQ_DH'])
    #Tvl = np.array(df['T_ah_hot_in'])
    #Tvls = np.array(df['T_ah_cold_out_zi'])
    x_data6 = np.array([Q,Tvl,Tvls])
    y_predicted = func(x_data6,popt[0],popt[1],popt[2],popt[3],popt[4])
    return y_predicted


In [40]:
def Heizungsneubau_aufbau(buildings_efh_nb,i=0,m=0):
    
    """ 
    this function uses the heizung_neubau_predictor return and builds the final profile of the return temperature

    Parameters:
    buildings_efh_nb: simulation data single family Building and Multi family Building 
    i: index of the simulation run
    m : parameter to choose between single family Building and Multi family Building 
    Return type:
    df : dataframe including the generated return temperatur with the column name y_prediction6
    accuracy : return an array fit goodnes measurments 
    
    Description : 
    call the predictor_speicher function to generate the profile of a simulation run with the fit parameter then replace
    the return temperature predicted for the near zero heating power with the supply temperature and generate the mass flow rate profile 

    """
    
    Datei = resampler_3auf15(buildings_efh_nb)
    null_index = null_leistung_index(Datei,min_leistung_kw = 0.02)

    #Datei = null_leistung(Datei,min_leistung_kw = 0.02)
    #Datei = alles(Datei,Datei[0].columns,get_iterator(buildings_efh,Ge = 0,step = 1))

    df = Datei[i]
    cp = 4181 /3600 #Wh/kg k
    def func(X, a, b, c,d,e):
        Q,Tvl,Tamb = X
        return a*Tvl**b*Q**c*Tamb**d+e

    popt_efh = [ 5.98956091e+00, -7.05859085e-02, -1.27534952e-02,  3.93690089e+00, 2.13355948e+01]

    popt_mfh = [ 5.50400318e+00, -1.08437222e-01, -3.86733584e-03,  4.35655705e+00, 2.16923621e+01]

    
    #popt_efh = [ 6.83048692e-06, -7.05369232e-02 ,-1.27490219e-02 , 3.93519206e+00, 2.13341953e+01]
    #popt_mfh = [ 1.64123514e-06 , -1.08427225e-01, -3.86862860e-03  ,4.35652380e+00,2.16923353e+01]
    
    popt = [popt_efh,popt_mfh]
    y_prediction6 = heizung_neubau_predictor(df,func,popt[m])
    df['y_prediction6'] = y_prediction6
    df['y_prediction6'].loc[null_index[i]] = df['T_ah_hot_in']
    df['dm_DH_zi'] = (df['dQ_DH']*1000)/(cp*(df['T_ah_hot_in']-df['y_prediction6']))  
    df['dm_DH_zi'].loc[null_index[i]] = 0
    MAE , MSE, r2,accuracy,ab = evaluate(np.array(df['T_ph_hot_out']),np.array(df['y_prediction6']))
    df['ab'] = np.array(ab)
    accuracy[1] = r2
    return df,accuracy

In [41]:
Heizung_neubau = Heizungsneubau_aufbau(buildings_efh_nb,i=0,m=0)

100%|█████████████████████████████████████████████████████████████████████████████████| 31/31 [00:00<00:00, 737.93it/s]

The model performance for testing set
--------------------------------------
MAE is 0.26971839771203177
MSE is 2.3617562753235997
MAPE is 0.011181987585885058
R2 score is 0.9954079693518588
95% percent of the data lay between ± 1.02


# Total System

In [42]:
'''
in this section the return temperature profile of the all four systems will be generated and the mixing of the two streams will
be calculated in order to generate the 


general commands:
m = 0 for single family houses 
m = 1 for multiple family houses

EFH_map[supply temp between 70 to 129] for single family houses 
MFH_map[supply temp between 70 to 129] for multple family houses

buildings_efh for single family houses 
buildings_mfh for multiple family houses

i = 0 == 70°C Tvlp to i = 30 == 100°C Tvlp
'''



EFH_map = map_temp_i(buildings_inst,Ge=0,step=2)
MFH_map = map_temp_i(buildings_inst,Ge=1,step=2)


In [43]:
TWW_DF ,accuracy  = prediktor_df(buildings_inst,i=EFH_map[70],m=0) #m =0 :EFH ;m = 1 :MFH , i = 0 =EFH 70,i = 1 = MFH 70
print(buildings_inst[EFH_map[70]]['building_name'][0])

100%|████████████████████████████████████████████████████████████████████████| 35041/35041 [00:00<00:00, 473425.59it/s]


The model performance for testing set
--------------------------------------
MAE is 1.0763249747269414
MSE is 30.069054215201007
MAPE is 0.08115170281585396
R2 score is 0.7590757226566895
95% percent of the data lay between ± 4.12
000-70-B_1.outr.zip


In [44]:
TWW_SP ,accuracy  = speicher_aufbau(buildings_str,i=EFH_map[70],m=0) #m =0 :EFH ;m = 1 :MFH , i = 0 =EFH 70,i = 1 = MFH 70
print(buildings_str[EFH_map[70]]['building_name'][0])

100%|███████████████████████████████████████████████████████████████████████████████| 120/120 [00:00<00:00, 398.58it/s]


The model performance for testing set
--------------------------------------
MAE is 0.9549178441649306
MSE is 2.042149993217819
MAPE is 0.017600306274105477
R2 score is 0.9439008433983953
95% percent of the data lay between ± 1.65
MFH_Storage_70.dck-2201261853\000-70-B_1.outr.zip


In [45]:
Heizung_bestand ,accuracy  = Heizung_aufbau(buildings_efh,i = 0,m =0 ) #m =0 :EFH ;m = 1 :MFH , i = 0 == 70 Tvlp

100%|████████████████████████████████████████████████████████████████████████████████| 60/60 [00:00<00:00, 1071.20it/s]

The model performance for testing set
--------------------------------------
MAE is 0.9407862792282954
MSE is 7.292936705454507
MAPE is 0.03318257668974864
R2 score is 0.9796128870482759
95% percent of the data lay between ± 3.34


In [46]:
Heizung_neubau ,accuracy  = Heizungsneubau_aufbau(buildings_efh_nb,i=0,m=0) ##m =0 :EFH ;m = 1 :MFH , i = 0 == 70 Tvlp

100%|█████████████████████████████████████████████████████████████████████████████████| 31/31 [00:00<00:00, 755.95it/s]

The model performance for testing set
--------------------------------------
MAE is 0.26971839771203177
MSE is 2.3617562753235997
MAPE is 0.011181987585885058
R2 score is 0.9954079693518588
95% percent of the data lay between ± 1.02


In [47]:
def Mishung(TWW,Heizung,rucklauf,flow): 

    """ 
    this functions calculate the mixing of the two mass flow rates for domestic water heating (TWW) and Heating system (Heizung)
    

    Parameters:
    TWW: Data for domestic water heating 
    Heizung: Data for heating system 
    rucklauf : the label for the return temperature in the TWW or Heizung dataframes 
    flow : the label for the mass flow rate in the TWW or Hezung dataframes
    
    Return type:
    Trl_m : profile of the mixtures return temperature 
    dm_m : profile of the mixtures mass flow rate 
    
    Description : 
    calculate the mixtures return temperature and mass flow rate according to the equation of the mixing of two mass flows. 
    
    """
    
    mdot_min_TWW = TWW[flow].sort_values().unique()[1]         #finding the minimum mass flow rate for domestic heating
    mdot_min_Heizung = Heizung[flow].sort_values().unique()[1]  # finding the minimum mass flow rate for the Heating System
    
    #initilaising of variables 
    iterator = np.arange(len(TWW))
    nom = np.arange(len(TWW))   
    den = np.arange(len(TWW))
    Trl_m = np.arange(len(TWW))
    dm_m =  np.arange(len(TWW))
    
    
    for i in tqdm(iterator):
        
        dm_m[i] = TWW[flow][i] + Heizung[flow][i]    #calculate the mixture flow rate 
        if (TWW[flow][i] < mdot_min_TWW and Heizung[flow][i] > mdot_min_Heizung)  : #kein TWW Bedarf, no domestic water heating demand 
            Trl_m[i] = Heizung[rucklauf][i] #mixture return temperature is equal to the heating system return temp

        if (Heizung[flow][i] < mdot_min_Heizung and TWW[flow][i] > mdot_min_TWW) : #kein Heizungsbedarf, no Room Heating demand
            Trl_m[i] = TWW[rucklauf][i] #mixture return temperature is equal to the domestic water heating system return temp

        if (Heizung[flow][i] < mdot_min_Heizung and TWW[flow][i] < mdot_min_TWW): #kein Bedarf bei den Beiden, no demand at all
            Trl_m[i] = TWW[rucklauf][i] #mixture return temperature is equal to the domestic water heating system return temp
            print(TWW[rucklauf][i])
        else : 
            nom[i] = (TWW[rucklauf][i]*TWW[flow][i])+(Heizung[rucklauf][i]*Heizung[flow][i]) #Normal Betrieb, normal demand 
            den[i] = TWW[flow][i]+Heizung[flow][i]+1      # the added one unit is to avoid the devision by zero 
            Trl_m[i] =  nom[i]/den[i]
            
    return Trl_m , dm_m

In [52]:
Trl_original, dm_m_original = Mishung(TWW_DF,Heizung_neubau,'T_ph_hot_out','dm_DH')  #generating the return temperature of the mix 
                                                                                     #for the simulation data 
Trl_prediction, dm_m_prediction = Mishung(TWW_DF,Heizung_neubau,'y_prediction6','dm_DH_zi') ##generating the return temperature of the mix 
                                                                                     #for the predicted data 
Trl_original2, dm_m_original2  = Mishung(TWW_SP,Heizung_neubau,'T_ph_hot_out','dm_DH')
Trl_prediction2, dm_m_prediction2  = Mishung(TWW_SP,Heizung_neubau,'y_prediction6','dm_DH_zi')

 18%|█████████████▍                                                            | 6352/35041 [00:00<00:00, 31761.71it/s]

70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0


 36%|██████████████████████████▏                                              | 12598/35041 [00:00<00:00, 29085.55it/s]

70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0


 52%|█████████████████████████████████████▊                                   | 18165/35041 [00:00<00:00, 24851.62it/s]

...


70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0

 78%|█████████████████████████████████████████████████████████                | 27419/35041 [00:01<00:00, 22878.82it/s]


70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0
70.0

100%|█████████████████████████████████████████████████████████████████████████| 35041/35041 [00:01<00:00, 25854.73it/s]

70.0
70.0
70.0
70.0


In [53]:
df = TWW_DF

df['or'] = Trl_original      # original gemischete rücklauftemperatur 
df['or'] = df['or'].replace(0,df['T_ah_hot_in'][0])
df['pre'] = Trl_prediction    # vorhergesagte  gemischete rücklauftemperatur
df['dm_DH_orignial'] = dm_m_original    # Original gemischeter Massenstrom 
df['dm_DH_prediction'] = dm_m_prediction  # vorhergesagte gemischeter Massenstrom


df['or2'] = Trl_original2
df['pre2'] = Trl_prediction2
df['dm_DH_orignial2'] = dm_m_original2
df['dm_DH_prediction2'] = dm_m_prediction2

In [54]:
MAE , MSE, r21,accuracy1,ab1 = evaluate(np.array(df['or']),np.array(df['pre']))
MAE , MSE, r22,accuracy2,ab2 = evaluate(np.array(df['or2']),np.array(df['pre2']))


The model performance for testing set
--------------------------------------
MAE is 0.8610484860591878
MSE is 2.516994378014326
MAPE is 0.024189258412646592
R2 score is 0.9892553485636755
95% percent of the data lay between ± 3.00
The model performance for testing set
--------------------------------------
MAE is 1.0382694557803716
MSE is 2.4814931080733995
MAPE is 0.023383362983317182
R2 score is 0.9843355970680507
95% percent of the data lay between ± 3.00
